# TIME ZONE CODE

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
from selenium import webdriver

## Airports Time scraping


The data of the time zone registred for each city is obtained from the web Zeitverschiebung ('https://www.zeitverschiebung.net/'), where it could be calculated the time difference between departure and arrival cities.

First of all, we will get the airports time zone data from the API of Aviation Edge Database ('https://aviation-edge.com/').

In [18]:
#API route
aed_city = 'https://aviation-edge.com/v2/public/cityDatabase?key=25034e-9edde7&codeIataCity='
#Flight to be requested (example)
city_code = 'MAD'
url = (f'{aed_city}{city_code}')
print(url)
html_city = requests.get(url).json()
html_city

https://aviation-edge.com/v2/public/cityDatabase?key=25034e-9edde7&codeIataCity=MAD


[{'GMT': '1',
  'cityId': 4421,
  'codeIataCity': 'MAD',
  'codeIso2Country': 'ES',
  'geonameId': 3117735,
  'latitudeCity': 40.416691,
  'longitudeCity': -3.700345,
  'nameCity': 'Madrid',
  'timezone': 'Europe/Madrid'}]

In [19]:
html_city_dic = html_city[0]
html_city_dic

{'GMT': '1',
 'cityId': 4421,
 'codeIataCity': 'MAD',
 'codeIso2Country': 'ES',
 'geonameId': 3117735,
 'latitudeCity': 40.416691,
 'longitudeCity': -3.700345,
 'nameCity': 'Madrid',
 'timezone': 'Europe/Madrid'}

In [28]:
city_df = pd.DataFrame(html_city_dic, index=[0])
city_df

,GMT,cityId,codeIataCity,codeIso2Country,geonameId,latitudeCity,longitudeCity,nameCity,timezone
0,1,4421,MAD,ES,3117735,40.416691,-3.700345,Madrid,Europe/Madrid


In [35]:
timezone = city_df.loc[0,'timezone']
print(timezone)

Europe/Madrid


Afterwards, we need to do web scraping in Zeitverschiebung to get the time difference between cities.

In [29]:
import os

os.environ['PATH'] = f'{os.environ["PATH"]}:{os.getcwd()}/drivers'
    
os.environ['PATH']

'/home/viki/miniconda3/envs/ironhack_env/bin:/home/viki/miniconda3/condabin:/home/viki/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin:/home/viki/Bootcamp/Destiny-tips-tool/drivers'

In [30]:
driver = webdriver.Chrome('/home/viki/Bootcamp/drivers/chromedriver')
driver.get('https://www.weatherwx.com/forecast.php?config=&forecast=pass&pass=tafINT')

For the scraping of this web, selenium is used as this is easier to program all the process.

In [111]:
import os

os.environ['PATH'] = f'{os.environ["PATH"]}:{os.getcwd()}/drivers'
    
os.environ['PATH']

'/home/viki/miniconda3/envs/ironhack_env/bin:/home/viki/miniconda3/condabin:/home/viki/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin:/home/viki/Bootcamp/Destiny-tips-tool/drivers:/home/viki/Bootcamp/Destiny-tips-tool/drivers:/home/viki/Bootcamp/Destiny-tips-tool/drivers:/home/viki/Bootcamp/Destiny-tips-tool/drivers'

In [32]:
driver = webdriver.Chrome('/home/viki/Bootcamp/drivers/chromedriver')
driver.get('https://www.zeitverschiebung.net/es/')

In [33]:
#Click cookies button
cookies_button = driver.find_element_by_class_name("css-flk0bs")
driver.execute_script("arguments[0].click();", cookies_button)


In [114]:
#Introduce departure time zone in first place
driver.find_element_by_xpath("//select[@name='Country']/option[text()='Spain']").click()

In [115]:
#Choose from Place select drop-down
driver.find_element_by_xpath("//select[@name='Places']/option[text()='Madrid / Barajas']").click()

In [116]:
#Click display forecast button
display_button = driver.find_element_by_class_name("btn.btn-default.btn-sm")

display_button.click()

In [117]:
#Get the information of the weather in 7 days, and introduce it in a list
table = driver.find_element_by_class_name('panel.panel-primary.article')

rows = table.find_elements_by_tag_name('table')

weather_raw = []
for row in rows:
    cells = row.find_elements_by_tag_name('td')
    weather_raw.append(cells[1].text)

print(weather_raw)

['Overcast Monday, temperatures as high as 19°C, low temperature around 10°C, wind out of the ESE 12 km/h. Barometric pressure 1017 mb, visibility 10 km, humidity 44%.', 'Patchy rain possible Tuesday, temperatures as high as 19°C, low temperature around 11°C, probability of precipitation 84%, wind out of the ESE 10 km/h. Barometric pressure 1013 mb', 'Patchy rain possible Wednesday, temperatures as high as 20°C, low temperature around 10°C, probability of precipitation 87%, wind out of the SSE 12 km/h. Barometric pressure 1012 mb', 'Patchy rain possible Thursday, temperatures as high as 20°C, low temperature around 12°C, probability of precipitation 80%, wind out of the S 10 km/h. Barometric pressure 1014 mb', 'Patchy rain possible Friday, temperatures as high as 16°C, low temperature around 12°C, probability of precipitation 84%, wind out of the S 18 km/h. Barometric pressure 1016 mb', 'Moderate rain Saturday, temperatures as high as 17°C, low temperature around 12°C, probability of p

In [118]:
#Divide the list in a list of list, by days
weather_raw_list = [[x] for x in weather_raw]
print(weather_raw_list)

[['Overcast Monday, temperatures as high as 19°C, low temperature around 10°C, wind out of the ESE 12 km/h. Barometric pressure 1017 mb, visibility 10 km, humidity 44%.'], ['Patchy rain possible Tuesday, temperatures as high as 19°C, low temperature around 11°C, probability of precipitation 84%, wind out of the ESE 10 km/h. Barometric pressure 1013 mb'], ['Patchy rain possible Wednesday, temperatures as high as 20°C, low temperature around 10°C, probability of precipitation 87%, wind out of the SSE 12 km/h. Barometric pressure 1012 mb'], ['Patchy rain possible Thursday, temperatures as high as 20°C, low temperature around 12°C, probability of precipitation 80%, wind out of the S 10 km/h. Barometric pressure 1014 mb'], ['Patchy rain possible Friday, temperatures as high as 16°C, low temperature around 12°C, probability of precipitation 84%, wind out of the S 18 km/h. Barometric pressure 1016 mb'], ['Moderate rain Saturday, temperatures as high as 17°C, low temperature around 12°C, proba

In [119]:
#Divide each sublist by the different weather features
weather_raw_divided = [re.split('[,.]', y) for x in weather_raw_list for y in x]
print(weather_raw_divided)

[['Overcast Monday', ' temperatures as high as 19°C', ' low temperature around 10°C', ' wind out of the ESE 12 km/h', ' Barometric pressure 1017 mb', ' visibility 10 km', ' humidity 44%', ''], ['Patchy rain possible Tuesday', ' temperatures as high as 19°C', ' low temperature around 11°C', ' probability of precipitation 84%', ' wind out of the ESE 10 km/h', ' Barometric pressure 1013 mb'], ['Patchy rain possible Wednesday', ' temperatures as high as 20°C', ' low temperature around 10°C', ' probability of precipitation 87%', ' wind out of the SSE 12 km/h', ' Barometric pressure 1012 mb'], ['Patchy rain possible Thursday', ' temperatures as high as 20°C', ' low temperature around 12°C', ' probability of precipitation 80%', ' wind out of the S 10 km/h', ' Barometric pressure 1014 mb'], ['Patchy rain possible Friday', ' temperatures as high as 16°C', ' low temperature around 12°C', ' probability of precipitation 84%', ' wind out of the S 18 km/h', ' Barometric pressure 1016 mb'], ['Moderat

In [120]:
'''
Function to order the data as we need and fill the empty data with 0. 
This will help when the data will be introduce in a dataset 
The data will be ordered as follows:
1 - Forecast
2 - High temperature
3 - Low temperature
4 - Probability of precipitation
5 - Wind
6 - Barometric pressure



'''

def reorder_func(day_weather_list):
    reordered_weather = [day_weather_list[0], day_weather_list[1], day_weather_list[2]]
    for i in range(2, len(day_weather_list)):
        if 'precipitation' in day_weather_list[i]:
            reordered_weather.append(day_weather_list[i])
    if len(reordered_weather)< 4:
        reordered_weather.append('0')

    for i in range(2, len(day_weather_list)):
        if 'wind' in day_weather_list[i]:
            reordered_weather.append(day_weather_list[i])
    if len(reordered_weather)< 5:
        reordered_weather.append('0')

    for i in range(2, len(day_weather_list)):
        if 'pressure' in day_weather_list[i]:
            reordered_weather.append(day_weather_list[i])
    if len(reordered_weather)< 6:
        reordered_weather.append('0')

    return reordered_weather

In [121]:
ordered_weather =[]
for x in weather_raw_divided:
    sublist = reorder_func(x)
    ordered_weather.append(sublist)
print(ordered_weather)

[['Overcast Monday', ' temperatures as high as 19°C', ' low temperature around 10°C', '0', ' wind out of the ESE 12 km/h', ' Barometric pressure 1017 mb'], ['Patchy rain possible Tuesday', ' temperatures as high as 19°C', ' low temperature around 11°C', ' probability of precipitation 84%', ' wind out of the ESE 10 km/h', ' Barometric pressure 1013 mb'], ['Patchy rain possible Wednesday', ' temperatures as high as 20°C', ' low temperature around 10°C', ' probability of precipitation 87%', ' wind out of the SSE 12 km/h', ' Barometric pressure 1012 mb'], ['Patchy rain possible Thursday', ' temperatures as high as 20°C', ' low temperature around 12°C', ' probability of precipitation 80%', ' wind out of the S 10 km/h', ' Barometric pressure 1014 mb'], ['Patchy rain possible Friday', ' temperatures as high as 16°C', ' low temperature around 12°C', ' probability of precipitation 84%', ' wind out of the S 18 km/h', ' Barometric pressure 1016 mb'], ['Moderate rain Saturday', ' temperatures as h

In [134]:
#Introduce all the data in a dataset
cols = ['Forecast', 'High temperature (ºC)', 'Low temperature (ºC)', 
        'Probability of precipitation (%)', 'Wind', 'Barometric pressure (mb)']
weather_df = pd.DataFrame(ordered_weather, columns = cols)
display(weather_df)

,Forecast,High temperature (ºC),Low temperature (ºC),Probability of precipitation (%),Wind,Barometric pressure (mb)
0,Overcast Monday,temperatures as high as 19°C,low temperature around 10°C,0,wind out of the ESE 12 km/h,Barometric pressure 1017 mb
1,Patchy rain possible Tuesday,temperatures as high as 19°C,low temperature around 11°C,probability of precipitation 84%,wind out of the ESE 10 km/h,Barometric pressure 1013 mb
2,Patchy rain possible Wednesday,temperatures as high as 20°C,low temperature around 10°C,probability of precipitation 87%,wind out of the SSE 12 km/h,Barometric pressure 1012 mb
3,Patchy rain possible Thursday,temperatures as high as 20°C,low temperature around 12°C,probability of precipitation 80%,wind out of the S 10 km/h,Barometric pressure 1014 mb
4,Patchy rain possible Friday,temperatures as high as 16°C,low temperature around 12°C,probability of precipitation 84%,wind out of the S 18 km/h,Barometric pressure 1016 mb
5,Moderate rain Saturday,temperatures as high as 17°C,low temperature around 12°C,probability of precipitation 76%,wind out of the ESE 16 km/h,Barometric pressure 1016 mb
6,Patchy rain possible Sunday,temperatures as high as 18°C,low temperature around 11°C,probability of precipitation 89%,wind out of the SSE 11 km/h,Barometric pressure 1015 mb


In [135]:
#Clean the data in Forecast
def not_day(text):
    short_text = ' '.join(text.split(' ')[:-1])
    return short_text
weather_df['Forecast'] = weather_df['Forecast'].apply(not_day)

In [136]:
#Clean High temperature, Low temperature, Probability of precipitation and Barometric pressure
#Keeping only numbers
def only_num(text):
    result = (re.findall('\d+', text))
    return ''.join(result)

weather_df['High temperature (ºC)'] = weather_df['High temperature (ºC)'].apply(only_num)
weather_df['Low temperature (ºC)'] = weather_df['Low temperature (ºC)'].apply(only_num)
weather_df['Probability of precipitation (%)'] = weather_df['Probability of precipitation (%)'].apply(only_num)
weather_df['Barometric pressure (mb)'] = weather_df['Barometric pressure (mb)'].apply(only_num)
display(weather_df)

,Forecast,High temperature (ºC),Low temperature (ºC),Probability of precipitation (%),Wind,Barometric pressure (mb)
0,Overcast,19,10,0,wind out of the ESE 12 km/h,1017
1,Patchy rain possible,19,11,84,wind out of the ESE 10 km/h,1013
2,Patchy rain possible,20,10,87,wind out of the SSE 12 km/h,1012
3,Patchy rain possible,20,12,80,wind out of the S 10 km/h,1014
4,Patchy rain possible,16,12,84,wind out of the S 18 km/h,1016
5,Moderate rain,17,12,76,wind out of the ESE 16 km/h,1016
6,Patchy rain possible,18,11,89,wind out of the SSE 11 km/h,1015
